In [ ]:
!pip install datasets

In [ ]:
from sklearn.model_selection import train_test_split
from moviepy.editor import VideoFileClip
from keras import layers, applications
from datasets import load_dataset
from tensorflow import keras
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2, os, gc

In [ ]:
ds = load_dataset("nexar-ai/nexar_collision_prediction", split='train')

Resolving data files:   0%|          | 0/1502 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1348 [00:00<?, ?it/s]

In [ ]:
n = round(ds.num_rows * 0.4)
subset = ds.shuffle(seed=1404).select(range(n))

In [ ]:
train_df = subset.to_pandas()

In [ ]:
train_df.head()

,video,time_of_event,time_of_alert,light_conditions,weather,scene,time_to_accident
0,"{'bytes': None, 'path': '/root/.cache/huggingf...",NaN,NaN,Normal,Cloudy,Sub-urban,NaN
1,"{'bytes': None, 'path': '/root/.cache/huggingf...",19.132,17.206,Normal,Clear,Urban,NaN
2,"{'bytes': None, 'path': '/root/.cache/huggingf...",20.167,18.300,Twilight,Clear,Sub-urban,NaN
3,"{'bytes': None, 'path': '/root/.cache/huggingf...",20.800,19.400,Normal,Rain,Urban,NaN
4,"{'bytes': None, 'path': '/root/.cache/huggingf...",NaN,NaN,Normal,Clear,Highway,NaN


In [ ]:
train_df.describe()

,time_of_event,time_of_alert,time_to_accident
count,292.000000,292.000000,0.0
mean,19.109010,17.542425,NaN
std,3.725076,3.808927,NaN
min,3.921000,3.072000,NaN
25%,19.095250,17.394500,NaN
50%,19.700000,18.181500,NaN
75%,20.253250,18.933000,NaN
max,56.800000,55.467000,NaN


In [ ]:
train_df.light_conditions.value_counts()

,count
light_conditions,
Normal,542
Twilight,37
Dark,16
Bright,5


In [ ]:
train_df.weather.value_counts()

,count
weather,
Clear,374
Cloudy,183
Rain,40
Snow,1


In [ ]:
train_df.scene.value_counts()

,count
scene,
Urban,304
Highway,157
Sub-urban,117
Other,10
Rural,7
Industrial,5


In [ ]:
train_df.time_of_event.isnull().sum() # half of the data is without accident (no augmentation needed)

np.int64(308)

In [ ]:
for col in train_df.columns:
    print(col, train_df[col].isnull().sum())

video 0
time_of_event 308
time_of_alert 308
light_conditions 0
weather 2
scene 0
time_to_accident 600


In [ ]:
train_df.video[0]

{'bytes': None,
 'path': '/root/.cache/huggingface/hub/datasets--nexar-ai--nexar_collision_prediction/snapshots/aa97deda5a59f00bb7187739053b7c72e14374df/train/negative/01041.mp4'}

In [ ]:
data = []
for idx, row in train_df.iterrows():
    video_path = row["video"]["path"]
    time_of_event = row.get("time_of_event", None)
    label = 0 if pd.isna(time_of_event) or time_of_event is None else 1
    data.append({
        "video_path": video_path,
        "label": label,
        "time_of_event": time_of_event
    })

video_df = pd.DataFrame(data)

In [ ]:
# === CONFIG ===
frame_interval = 8                 # every 8th frame
output_dir = "video_matrices"      # folder for extracted numpy arrays
os.makedirs(output_dir, exist_ok=True)

# === Extract event-centered clips using OpenCV ===
for idx, row in tqdm(video_df.iterrows(), total=len(video_df)):
    video_path = row["video_path"]
    time_of_event = row["time_of_event"]

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"[WARN] Cannot open video at index {idx}: {video_path}")
        continue

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0 or fps is None:
        print(f"[WARN] FPS is zero at index {idx}")
        cap.release()
        continue

    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = total_frames / fps

    # Define clip range (in seconds)
    if pd.isna(time_of_event) or time_of_event is None:
        start_time = 0
        end_time = min(duration, 5)
    else:
        start_time = max(0, time_of_event - 2)
        end_time = min(duration, time_of_event + 3)

    # Convert to frame indices
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    frames_list = []
    frame_index = start_frame

    while frame_index <= end_frame:
        success, frame = cap.read()
        if not success:
            break
        if frame_index % frame_interval == 0:
            frame_resized = cv2.resize(frame, (224, 224))
            frames_list.append(frame_resized)
        frame_index += 1

    cap.release()

    if frames_list:
        video_matrix = np.stack(frames_list)  # shape: (N, 224, 224, 3)
        save_path = os.path.join(output_dir, f"video_{idx}.npy")
        np.save(save_path, video_matrix)

    del frames_list
    gc.collect()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-2148926751.py", line 1, in <cell line: 0>
    from moviepy.editor import VideoFileClip
  File "/usr/local/lib/python3.12/dist-packages/moviepy/editor.py", line 110, in <module>
    from moviepy.video.io.preview import show, preview
  File "/usr/local/lib/python3.12/dist-packages/moviepy/video/io/preview.py", line 6, in <module>
    import pygame as pg
  File "/usr/local/lib/python3.12/dist-packages/pygame/__init__.py", line 291, in <module>
    import pygame.pkgdata
  File "/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py", line 25, in <module>
    from pkg_resources import resource_stream, resource_exists
  File "/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py", line 95, in <module>
    from jaraco.text import drop_comment, join_continuatio

TypeError: object of type 'NoneType' has no len()

In [ ]:
output_dir = "video_matrices"
train_df["video_matrix_path"] = None

for idx in train_df.index:
    file_path = os.path.join(output_dir, f"video_{idx}.npy")
    if os.path.exists(file_path):
        train_df.at[idx, "video_matrix_path"] = file_path
    else:
        train_df.at[idx, "video_matrix_path"] = None


In [ ]:
train_df.drop('time_to_accident', axis=1, inplace=True)

In [ ]:
train_df['label'] = video_df.label

In [ ]:
train_df = pd.get_dummies(
    train_df,
    columns=['light_conditions', 'weather', 'scene'],
    prefix=['light', 'weather', 'scene'],
    dtype=int
)

In [ ]:
train_df = train_df.drop(columns=['video', 'time_of_event', 'time_of_alert'])

In [ ]:
MAX_FRAMES = 20
def pad_or_truncate(video, max_frames=MAX_FRAMES):
    num_frames = video.shape[0]
    if num_frames >= max_frames:
        return video[:max_frames]
    else:
        pad = np.repeat(video[-1][np.newaxis, ...], max_frames - num_frames, axis=0)
        return np.concatenate([video, pad], axis=0)

class VideoMatrixSequence(keras.utils.Sequence):
    def __init__(self, df, batch_size=10, target_size=(224, 224), shuffle=True):
        self.df = df.reset_index(drop=True)
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.columns = self.df.columns.drop(['video_matrix_path', 'label'])
        self.indices = np.arange(len(self.df))
        self.on_epoch_end()

    def __len__(self):
        return len(self.df) // self.batch_size

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size : (idx + 1) * self.batch_size]

        tabular_data = []
        video_data = []
        labels = []

        for i in batch_indices :
            row = self.df.iloc[i]
            video = np.load(row["video_matrix_path"])

            frames_resized = [cv2.resize(frame, self.target_size) for frame in video]
            frames_resized = np.stack(frames_resized)
            frames_resized = pad_or_truncate(frames_resized, MAX_FRAMES)

            tabular_features = [row[col] for col in self.columns]
            tabular_data.append(tabular_features)

            video_data.append(frames_resized)
            labels.append(float(row["label"]))

        tabular_array = np.array(tabular_data, dtype=np.float32)
        video_array = np.array(video_data, dtype=np.float32)

        labels_array = np.array(labels, dtype=np.float32)
        labels_array = np.expand_dims(labels_array, axis=-1)

        return (video_array, tabular_array), labels_array

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)


In [ ]:
train_df_split, val_df_split = train_test_split(train_df, test_size=0.3, random_state=1404)

train_seq = VideoMatrixSequence(train_df_split, batch_size=16, target_size=(224, 224))
val_seq = VideoMatrixSequence(val_df_split, batch_size=16, target_size=(224, 224))

In [ ]:
IMG_SIZE = 224
SEQ_LEN = 20
NUM_TABULAR_FEATURES = 14

video_input = keras.Input(shape=(MAX_FRAMES, IMG_SIZE, IMG_SIZE, 3), name="video")
tabular_input = keras.Input(shape=(NUM_TABULAR_FEATURES,), name="tabular")

base_cnn = applications.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    pooling='avg',
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_cnn.trainable = False

video_features = layers.TimeDistributed(base_cnn)(video_input)

lstm_out = layers.Bidirectional(layers.LSTM(512))(video_features)
video_final = layers.Dropout(0.6)(lstm_out)

tabular_dense = layers.Dense(64, activation='relu')(tabular_input)
tabular_final = layers.Dropout(0.3)(tabular_dense)

merged = layers.concatenate([video_final, tabular_final])

outputs = layers.Dense(1, activation='sigmoid', name="output")(merged)

model = keras.Model(inputs=[video_input, tabular_input], outputs=outputs, name="video_tabular_resnet_gru")

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5, clipnorm=1.0),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.FBetaScore(beta=1.0, average='micro', name='f1'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall')
    ]
)

cbs = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=8,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="best_video_model.keras",
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

history = model.fit(train_seq, validation_data=val_seq, epochs=15, callbacks=cbs)

  self._warn_if_super_not_called()



Epoch 1/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 772ms/step - accuracy: 0.5530 - f1: 0.6314 - loss: 0.6918 - precision: 0.5173 - recall: 0.5019
Epoch 1: val_accuracy improved from -inf to 0.57955, saving model to best_video_model.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 266s 3s/step - accuracy: 0.5537 - f1: 0.6317 - loss: 0.6916 - precision: 0.5184 - recall: 0.5014 - val_accuracy: 0.5795 - val_f1: 0.6866 - val_loss: 0.6824 - val_precision: 0.6552 - val_recall: 0.4130
Epoch 2/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 772ms/step - accuracy: 0.6009 - f1: 0.6260 - loss: 0.6562 - precision: 0.5788 - recall: 0.4584
Epoch 2: val_accuracy improved from 0.57955 to 0.63068, saving model to best_video_model.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.6018 - f1: 0.6265 - loss: 0.6562 - precision: 0.5809 - recall: 0.4594 - val_accuracy: 0.6307 - val_f1: 0.6963 - val_loss: 0.6494 - val_precision: 0.6883 - val_recall: 0.5638
Epoch 3/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 772ms/step - accuracy: 0.6782 - f1: 0.6315 - 